In [1]:
pip --version

pip 24.0 from /Users/vihar_gunamgari/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pip (python 3.11)
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install torch transformers pyserini numpy pandas matplotlib


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install datasets
from datasets import load_dataset

dataset_python = load_dataset("code_search_net", "python", trust_remote_code=True)
dataset_java = load_dataset("code_search_net", "java", trust_remote_code=True)


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


/Users/vihar_gunamgari/.pyenv/versions/3.11.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pandas as pd

# Extract relevant fields
filtered_data_python = [
    {
        "id": i,
        "language": entry["language"],
        "function_name": entry["func_name"],
        "code": entry["func_code_string"],
        "docstring": entry["func_documentation_string"],
        "file_path": entry["func_path_in_repository"],
    }
    for i, entry in enumerate(dataset_python["train"])
]

# Convert to DataFrame
df_python = pd.DataFrame(filtered_data_python)

print(df_python.head())

   id language                function_name  \
0   0   python  AbstractElement.addidsuffix   
1   1   python   AbstractElement.setparents   
2   2   python       AbstractElement.setdoc   
3   3   python      AbstractElement.hastext   
4   4   python      AbstractElement.hasphon   

                                                code  \
0  def addidsuffix(self, idsuffix, recursive = Tr...   
1  def setparents(self):\n        """Correct all ...   
2  def setdoc(self,newdoc):\n        """Set a dif...   
3  def hastext(self,cls='current',strict=True, co...   
4  def hasphon(self,cls='current',strict=True,cor...   

                                           docstring                file_path  
0  Appends a suffix to this element's ID, and opt...  pynlpl/formats/folia.py  
1  Correct all parent relations for elements with...  pynlpl/formats/folia.py  
2  Set a different document. Usually no need to c...  pynlpl/formats/folia.py  
3  Does this element have text (of the specified ...  pynlpl

In [5]:
import pandas as pd

# Extract relevant fields
filtered_data_java = [
    {
        "id": i,
        "language": entry["language"],
        "function_name": entry["func_name"],
        "code": entry["func_code_string"],
        "docstring": entry["func_documentation_string"],
        "file_path": entry["func_path_in_repository"]
    }
    for i, entry in enumerate(dataset_java["train"])
]

# Convert to DataFrame
df_java = pd.DataFrame(filtered_data_java)

print(df_java.head())

   id language                           function_name  \
0   0     java       ProtectionContainersInner.inquire   
1   1     java  ProtectionContainersInner.inquireAsync   
2   2     java  ProtectionContainersInner.inquireAsync   
3   3     java       ProtectionContainersInner.refresh   
4   4     java  ProtectionContainersInner.refreshAsync   

                                                code  \
0  public void inquire(String vaultName, String r...   
1  public ServiceFuture<Void> inquireAsync(String...   
2  public Observable<Void> inquireAsync(String va...   
3  public void refresh(String vaultName, String r...   
4  public Observable<Void> refreshAsync(String va...   

                                           docstring  \
0  Inquires all the protectable item in the given...   
1  Inquires all the protectable item in the given...   
2  Inquires all the protectable item in the given...   
3  Discovers all the containers in the subscripti...   
4  Discovers all the containers in

In [6]:
# Clean and Normalize the code
# - remove excess spaces
# - remove comments (though Python docstrings can be useful)
# - convert everything to lowercase for consistency (except code syntax)

import re

def clean_python_code(code):
    if not isinstance(code, str):
        return ""
    code = re.sub(r'#.*', '', code) # remove inline comments
    code = re.sub(r'\s+', '', code).strip() # normalize spaces
    return code

def clean_java_code(code):
    if not isinstance(code, str):
        return ""
    code = re.sub(r'//.*', '', code) # remove single-line comments
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL) # remove multi-line comments
    code = re.sub(r'\s+', ' ', code).strip() # normalize spaces
    return code

df_python["clean_code"] = df_python["code"].apply(clean_python_code)
df_java["clean_code"] = df_java["code"].apply(clean_java_code)

In [7]:
# Normalize queries

def preprocess_query(query):
    query = query.lower().strip() # convert to lowercase
    query = re.sub(r'[^\w\s]', '', query) # remove special characters
    return query

sample_query = "Binary search in Python"
print(preprocess_query(sample_query)) # output: "binary search in python"

binary search in python


In [8]:
# Save processed data

df_python.to_json("preprocessed_codesearchnet_python.json", orient="records", indent=2)
df_java.to_json("preprocessed_codesearchnet_java.json", orient="records", indent=2)